# Keras的輔助工具-callbacks  
在這之前已經了解了如何訓練一個model，但是有時候如果model已經無法收斂，或者想要調整訓練時的learning rate該如何呢？  
這時候只要將**`callbacks`**參數加入`fit()`中就可以了。  
Keras有一些內建好的callbacks可以使用，像是：
+ ModelCheckpoint: 根據指定的評估方式來決定是否將model weights保留。  
+ EarlyStopping: 根據指定的評估方式來決定是否停止訓練。  
+ LearningRateScheduler: 可根據function定義learning rate，調整訓練時候的learning rate。  
+ ReduceLROnPlateau: 根據指定的評估方式調整訓練時的learning rate。
+ TensorBoard: 可以將訓練記錄成可視化的方法。  
+ ...

還有其他內建的callbacks，以上是比較常用的幾個，當然也可以自定義callbacks。  
這次以EarlyStopping與ReduceLROnPlateau為例子。  
更多callbacks訊息，請參閱[callbacks](https://keras.io/api/callbacks/)。  

In [1]:
# 載入所需lib
import numpy as np
import math
import time
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


In [2]:
def get_model():
    inputs = tf.keras.Input(shape=(28, 28, 1))

    # model layer
    conv_1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
    max_pool_1 = tf.keras.layers.MaxPooling2D()
    conv_2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')
    max_pool_2 = tf.keras.layers.MaxPooling2D()
    flatten = tf.keras.layers.Flatten()
    drop = tf.keras.layers.Dropout(0.5)
    output = tf.keras.layers.Dense(10, activation='softmax')

    # path
    x = conv_1(inputs)
    x = max_pool_1(x)
    x = conv_2(x)
    x = max_pool_2(x)
    x = flatten(x)
    x = drop(x)
    x = output(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.summary()
    return model

In [3]:
#download MNIST dataset and preprocessing
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255
y_train = y_train.astype('float32')
x_test = x_test.astype('float32') / 255
y_test = y_test.astype('float32')

x_train = np.expand_dims(x_train, -1)
y_train = np.expand_dims(y_train, -1)
x_test = np.expand_dims(x_test, -1)
y_test = np.expand_dims(y_test, -1)

## callbacks

In [4]:
# 用一個list存放選擇的callbacks
# 衡量指標設定要與fit顯示的名稱相同

'''
early stopping 設定
衡量指標為'val loss'
衡量指標變化小於0.001，則判定沒有改變
連續5次無改變則停止訓練
停止訓練並且將最好的val loss時的weights寫入目前model
發生更改的時候顯示訊息
'''
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=5,
    restore_best_weights=True,
    verbose=1)

'''
ReduceLROnPlateau
衡量指標為'val loss'
降低learning rate的權重為0.1，new learning rate = learning * factor
衡量指標變化小於0.001，則判定沒有改變
連續2次無改變則降低learning rate
learning rate最低降到1E-5
發生更改的時候顯示訊息
'''
reLR = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    min_delta=0.001,
    min_lr=1e-5,
    verbose=1)
# 將所有設定好的callbacks放入list
my_callbacks = [early_stop, reLR]

# parameter init
epochs = 100
batch_size = 128

# optimizer
opt = tf.keras.optimizers.Adam()

# loss
loss_fu = tf.keras.losses.SparseCategoricalCrossentropy()

# accuracy
acc = tf.keras.metrics.SparseCategoricalAccuracy()
model = get_model()
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy']
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0     

將`my_callbacks`放入`fit()`中

In [5]:
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=my_callbacks
)

Epoch 1/100
422/422 [==============================] - 2s 5ms/step - loss: 0.3584 - sparse_categorical_accuracy: 0.8930 - val_loss: 0.0842 - val_sparse_categorical_accuracy: 0.9767 - lr: 0.0010
Epoch 2/100
422/422 [==============================] - 2s 4ms/step - loss: 0.1135 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.0559 - val_sparse_categorical_accuracy: 0.9843 - lr: 0.0010
Epoch 3/100
422/422 [==============================] - 2s 4ms/step - loss: 0.0853 - sparse_categorical_accuracy: 0.9735 - val_loss: 0.0482 - val_sparse_categorical_accuracy: 0.9858 - lr: 0.0010
Epoch 4/100
422/422 [==============================] - 2s 4ms/step - loss: 0.0710 - sparse_categorical_accuracy: 0.9779 - val_loss: 0.0416 - val_sparse_categorical_accuracy: 0.9893 - lr: 0.0010
Epoch 5/100
422/422 [==============================] - 2s 4ms/step - loss: 0.0632 - sparse_categorical_accuracy: 0.9803 - val_loss: 0.0396 - val_sparse_categorical_accuracy: 0.9893 - lr: 0.0010
Epoch 6/100
422/422 [=========

可以看到原本設定要跑100個epoch，在地17個epoch時候就結束並且寫回最優weights，中間也因3次val loss判定連續沒有改變降低了learning rate，並且最後一次下降因為達到低標而設定在1E-5。  
其他的`callbacks`用法也是相同概念，了解該`callbacks`用途以及設定好參數，再放入`fit()`中就可以使用，真的很方便。  
當然，有些特殊需求也可以透過`自定義callbacks`來達成。

## **總結**  
到此，使用Keras建立model的基本方法算是結束，只要將以下幾點完成即可生成一個基本的model。  
+ 建立好模型的架構與定義好`input`與`output`
+ 設定好相關參數與function(optimizer、loss function、callbacks等等)  
+ 資料的前處理(ex:歸一化)與送入方式(ex:Sequence、tf.Data等等)  

當然，最新的一些技術(ex:新的loss function)提供的API可能無法達成，這時候就得`自定義方式`處理。  
接下來的主題將會環繞在`自定義`，也就是`Custom`的進階方式。